<div style='text-align: center;'>
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>COMP 555 Bioalgorithms Spring 2021</h1>
<h1 style='font-size: 200%;'>Problem Set 1</h1>
</div>

---
This problem set will examine the viral genome <a href="http://www.csbio.unc.edu/mcmillan/Comp555S21/SARS-CoV-2.fa" download="SARS_CoV-2.fa">SARS-CoV-2.fa</a>, the <a href="http://www.csbio.unc.edu/mcmillan/Comp555S21/SARS-CoV-2.fa" download="GCA_000001405.15_GRCh38_genomic.fna">human genome</a>, and the Long Terminal Repeat sequence from HERV-K, <a href="http://www.csbio.unc.edu/mcmillan/Comp555S21/LTR14A.fa" download="LTR14A.fa">LTR14A.fa</a>.

In [1]:
import gzip
import itertools
import math
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

def loadFasta(filename):
    """ Parses a classically formatted and possibly compressed
        FASTA file into two lists. One of headers and a second
        list of sequences. The ith index of each list correspond.
    """
    if filename.endswith('.gz'):
        fp = gzip.open(filename, 'r')
    else:
        fp = open(filename, 'r')
    # Split at headers
    data = fp.read().split('>')
    fp.close()
    # Ignore whatever appears before the first header
    data.pop(0)     
    headers = []
    sequences = []
    for sequence in data:
        lines = sequence.split('\n')
        headers.append(lines.pop(0))
        # Add an extra '+' to make string '1-referenced'
        sequences.append('+' + ''.join(lines))
    return (headers, sequences)

---
**Problem 1:** Write a function that produces a list of ***missing*** <em>k</em>-mers from a given sequence. If all possible *k*-mers are present, the function should return an empty list.

In [2]:
def missingKmers(dnaseq, k):
    missing  = []

    kmerSet = set()
    for i in range(1, len(dnaseq)-k+1):
        kmer = dnaseq[i:i+k]
        kmerSet.add(kmer)

    for pattern in itertools.product('ACGT', repeat=k):
        kmer = ''.join(pattern)
        if kmer not in kmerSet:
            missing.append(kmer)

    return missing

In [3]:
result = missingKmers('+GAGACATTAGACATAACC', 2)
# result = missingKmers('+GAGACATTAGACATAACCGCTCTGGT', 2)
print(result)

['CG', 'CT', 'GC', 'GG', 'GT', 'TC', 'TG']


---
**Problem 2:** Apply your function from **Problem 1** to the sequence from Human Chromosome 1 with a *k* value of 11. Examine the list of missing *k*-mers and find how many of the possible 11-mers are missing from Chromosome 1?

In [4]:
dna = open('data/HumanSeq/Chr1.seq', 'r').read()
print(len(dna))

248956423


In [5]:
result = missingKmers(dna, 11)
print(len(result), len(result)/(4**11))

147720 0.03521919250488281


---
**Problem 3:** Given *only* the list of missing 11-mers from **Problem 2**, predict how many 10-mers are missing. Also, using *only* these missing 11-mers, predict a lower-bound of the number of missing 12-mers in Chromosome 1.

In [6]:
missing10mers = []
for kmer in result:
    prefix = kmer[:-1]
    for extra in 'ACGT':
        if prefix + extra not in result:
            break
        if extra + prefix not in result:
            break
    else:
        if prefix not in missing10mers:
            missing10mers.append(prefix)
print(len(missing10mers))

2649


In [7]:
missing12mers = set()
for kmer in result:
    for extra in 'ACGT':
        missing12mers.add(kmer + extra)
        missing12mers.add(extra + kmer)
print(len(missing12mers))

954670


---
**Problem 4:** The goal of this problem is to make a list identifying potential full ERVs in the human genome. Write a code fragment that uses the *(position, kmer-index)* tuples from Lecture 4 to find all LTR pairs satisfying the following conditions:

1. Both LTRs include at least 20% of their expected *k*-mers in order.
2. Both LTRs are on the same strand.
3. The span from the first *k*-mer of the first LTR to the last *k*-mer of the second LTR is less than 10000 bases.

Your code should output a list of tuples with the following values in order, the contig name, the position of first *k*-mer of the first LRT, and the postion of the last *k*-mer of the second LTR. For example, the interval output for the example given in class would be:

<pre>('1', 62178464, 62183771)</pre>

In [8]:
def revComp(dnaSeq):
    return ''.join([{'A':'T',
                     'C':'G',
                     'G':'C',
                     'T':'A'}[base] for base in reversed(dnaSeq)])

In [9]:
hdr, seq = loadFasta('data/LTR14A.fa')

In [10]:
ltr = seq[0].upper()
K = 19
forward = dict([(ltr[i:i+K], i) for i in range(1, len(ltr)-K+1)])
print(len(forward))
rev = '+' + revComp(ltr[1:])
reverse = dict([(rev[i:i+K], -i) for i in range(1, len(rev)-K+1)])
print(len(reverse))

326


In [11]:
import time

chromo = [str(i) for i in range(1, 23)] + ['X', 'Y', 'MT']

genome = []
kmerCount = {}
for contig in chromo:
    tick = time.time()
    position = []
    with open('data/HumanSeq/Chr%s.seq' % contig, 'r') as fp:
        chrseq = fp.read()
    for i in range(1, len(chrseq)-K+1):
        kmer = chrseq[i:i+K]
        if kmer in forward:
            position.append((i, forward[kmer]))
        elif kmer in reverse:
            position.append((i, reverse[kmer]))
        else:
            if len(position) > 2 and position[-2][1] == 0 and position[-1][1] == 0:
                position.pop()
            position.append((i, 0))
    tock = time.time()
    print(contig, len(chrseq), len(position), '%6.2f secs' % (tock - tick))
    tick = tock
    genome.append(position)

1 248956423 1698 434.88 secs
2 242193530 1265 229.41 secs
3 198295560 1060 177.49 secs
4 190214556 786 194.33 secs
5 181538260 1243 211.90 secs
6 170805980 1393 139.57 secs
7 159345974 1301 125.63 secs
8 145138637 345 107.33 secs
9 138394718 511 190.24 secs
10 133797423 2181 243.09 secs
11 135086623 914 244.79 secs
12 133275310 638 242.45 secs
13 114364329 620 202.00 secs
14 107043719 209 204.46 secs
15 101991190 839 183.84 secs
16 90338346 173 165.92 secs
17 83257442 701 154.90 secs
18 80373286 288 140.82 secs
19 58617617 693 105.45 secs
20 64444168 118 112.76 secs
21 46709984 347  82.54 secs
22 50818469 924  92.93 secs
X 156040896 1665 275.12 secs
Y 57227416 391 101.92 secs
MT 16570 3   0.06 secs


In [12]:
def potentialLtrs(genomeIdx):
    start = end = -1
    startIdx = prevIdx = counter = 0
    possibleLtrs = set()
    
    for elt in genomeIdx:
        position, kmerIdx = elt[0], elt[1]
        
        if (startIdx > 0 and kmerIdx > prevIdx) or (startIdx < 0 and kmerIdx < prevIdx):
            counter += 1
            end = position
            prevIdx = kmerIdx
        
        elif kmerIdx != 0:
            if counter > 66 and end - start <= 326:
                possibleLtrs.add((strand, start, end))
            start = position
            startIdx = kmerIdx
            prevIdx = kmerIdx
            strand = 'NEGATIVE' if startIdx < 0 else 'POSITIVE'
            counter = 1

    return sorted(possibleLtrs)

In [13]:
ltrPairs = []
for chrom in range(len(genome)):
    possibleLtrs = potentialLtrs(genome[chrom])

    positiveStrand = list(filter(lambda x: x[0] == 'POSITIVE', possibleLtrs))
    negativeStrand = list(filter(lambda x: x[0] == 'NEGATIVE', possibleLtrs))

    for k in range(len(positiveStrand) - 1):
        if positiveStrand[k+1][2] - positiveStrand[k][1] < 10000:
            ltrPairs.append((str(chrom+1) if chrom < 22
                                          else 'X' if chrom == 22
                                          else 'Y' if chrom == 23
                                          else 'MT', positiveStrand[k][1], positiveStrand[k+1][2]))

    for k in range(len(negativeStrand) - 1):
        if negativeStrand[k+1][2] - negativeStrand[k][1] < 10000:
            ltrPairs.append((str(chrom+1) if chrom < 22
                                          else 'X' if chrom == 22
                                          else 'Y' if chrom == 23
                                          else 'MT', negativeStrand[k][1], negativeStrand[k+1][2]))            
print(ltrPairs)

[('1', 62178464, 62183771), ('6', 134600811, 134605405), ('7', 138339214, 138345790), ('17', 75022576, 75030502), ('22', 30675436, 30679905), ('X', 101183288, 101189830), ('X', 118293074, 118300698)]
